# Finance Project 

In [68]:
from collections import Counter, defaultdict
from itertools import combinations 
import pandas as pd
import numpy as np
import math as math
import random
import itertools
from sklearn import svm
from sklearn import cross_validation
import sklearn
from sklearn.tree import DecisionTreeClassifier
from sklearn.feature_extraction import DictVectorizer
from sklearn.cross_validation import cross_val_score
import random
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import precision_recall_fscore_support
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
import warnings
import matplotlib.pyplot as plt

In [69]:
warnings.filterwarnings("ignore")

# 1. Data Processing

In [70]:
## read in data to data frame and inspect
data = pd.read_csv('AAPL_05222012_0930_1300_LOB_2.csv')
data.head()

,Index,Time,TVITCH_41::AAPL.BID_PRICE1..TVITCH_41__AAPL_3,TVITCH_41::AAPL.BID_UPDATE_TIME1..TVITCH_41__AAPL_3,TVITCH_41::AAPL.BID_SIZE1..TVITCH_41__AAPL_3,TVITCH_41::AAPL.ASK_PRICE1..TVITCH_41__AAPL_3,TVITCH_41::AAPL.ASK_UPDATE_TIME1..TVITCH_41__AAPL_3,TVITCH_41::AAPL.ASK_SIZE1..TVITCH_41__AAPL_3,TVITCH_41::AAPL.BID_PRICE2..TVITCH_41__AAPL_3,TVITCH_41::AAPL.BID_UPDATE_TIME2..TVITCH_41__AAPL_3,...,TVITCH_41::AAPL.BID_SIZE9..TVITCH_41__AAPL_3,TVITCH_41::AAPL.ASK_PRICE9..TVITCH_41__AAPL_3,TVITCH_41::AAPL.ASK_UPDATE_TIME9..TVITCH_41__AAPL_3,TVITCH_41::AAPL.ASK_SIZE9..TVITCH_41__AAPL_3,TVITCH_41::AAPL.BID_PRICE10..TVITCH_41__AAPL_3,TVITCH_41::AAPL.BID_UPDATE_TIME10..TVITCH_41__AAPL_3,TVITCH_41::AAPL.BID_SIZE10..TVITCH_41__AAPL_3,TVITCH_41::AAPL.ASK_PRICE10..TVITCH_41__AAPL_3,TVITCH_41::AAPL.ASK_UPDATE_TIME10..TVITCH_41__AAPL_3,TVITCH_41::AAPL.ASK_SIZE10..TVITCH_41__AAPL_3
0,1,2012/05/22 09:30:00.000,569.02,2012/05/22 09:29:34.006,40,570,2012/05/22 09:29:43.573,400,568.80,2012/05/22 09:29:47.563,...,200,571.00,2012/05/22 09:29:49.424,120,567.16,2012/05/22 09:28:28.730,60,571.24,2012/05/22 08:13:36.797,200
1,2,2012/05/22 09:30:00.003,569.02,2012/05/22 09:29:34.006,40,570,2012/05/22 09:29:43.573,400,568.80,2012/05/22 09:29:47.563,...,200,570.99,2012/05/22 09:29:53.347,200,567.16,2012/05/22 09:28:28.730,60,571.00,2012/05/22 09:29:49.424,120
2,3,2012/05/22 09:30:00.003,569.02,2012/05/22 09:29:34.006,40,570,2012/05/22 09:29:43.573,400,568.80,2012/05/22 09:29:47.563,...,200,570.67,2012/05/22 09:29:29.034,10,567.16,2012/05/22 09:28:28.730,60,570.99,2012/05/22 09:29:53.347,200
3,4,2012/05/22 09:30:00.003,569.03,2012/05/22 09:30:00.003,8,570,2012/05/22 09:29:43.573,400,569.02,2012/05/22 09:29:34.006,...,200,570.67,2012/05/22 09:29:29.034,10,567.30,2012/05/22 09:29:38.985,200,570.99,2012/05/22 09:29:53.347,200
4,5,2012/05/22 09:30:00.003,569.03,2012/05/22 09:30:00.003,8,570,2012/05/22 09:29:43.573,400,569.02,2012/05/22 09:29:34.006,...,200,570.55,2012/05/22 09:30:00.003,8,567.30,2012/05/22 09:29:38.985,200,570.67,2012/05/22 09:29:29.034,10


In [71]:
## set aside test set 11:00-12:30
test = data[data["Time"] >= '2012/05/22 11:00:00.000']
test = test.reset_index(drop = True)
test.head()
len(test)

129324

In [72]:
## subset data, delta_t=30
def subset(delta_t = 30):
    sub = data[::delta_t]
    sub = sub.reset_index(drop = True)
    sub["Index"] = (sub["Index"]/delta_t).apply(math.ceil)
    return(sub)

In [73]:
data30 = subset()
data30.head()
len(data30)

11090

In [74]:
## get headers for indexing later
headers = list(data.columns.values)
print(headers[5:61:6])

['TVITCH_41::AAPL.ASK_PRICE1..TVITCH_41__AAPL_3', 'TVITCH_41::AAPL.ASK_PRICE2..TVITCH_41__AAPL_3', 'TVITCH_41::AAPL.ASK_PRICE3..TVITCH_41__AAPL_3', 'TVITCH_41::AAPL.ASK_PRICE4..TVITCH_41__AAPL_3', 'TVITCH_41::AAPL.ASK_PRICE5..TVITCH_41__AAPL_3', 'TVITCH_41::AAPL.ASK_PRICE6..TVITCH_41__AAPL_3', 'TVITCH_41::AAPL.ASK_PRICE7..TVITCH_41__AAPL_3', 'TVITCH_41::AAPL.ASK_PRICE8..TVITCH_41__AAPL_3', 'TVITCH_41::AAPL.ASK_PRICE9..TVITCH_41__AAPL_3', 'TVITCH_41::AAPL.ASK_PRICE10..TVITCH_41__AAPL_3']


In [75]:
## separate training and validation sets
train = data30[data30["Time"] < '2012/05/22 10:30:00.000']
train= train.reset_index(drop = True)
valid = data30[(data30["Time"] >= '2012/05/22 10:30:00.000') & (data30["Time"] < '2012/05/22 11:00:00.000')]
valid= valid.reset_index(drop = True)
len(train) + len(valid)

6779

In [76]:
def build_features(col):
    log_col = np.log(col)
    cum_avg_col = pd.rolling_mean(col, window = 10)
    cum_avg_col[0:11] = col[0:11]
    log_cum_avg_col = np.log(cum_avg_col)
    return(log_col, cum_avg_col, log_cum_avg_col)

In [77]:
def data_processing(data, headers, delta_t = 1):
    ## now create the features_df that contains only the features to train on 
    columns = ["Max_Bid","log_Max_Bid",'Spread_Ask_Col',"Min_Ask","log_Min_Ask","Mid_Price","log_Mid_Price",
               "Avg_Bid_Vol","log_Avg_Bid_Vol","Cum_Avg_Bid_Vol","Avg_Ask_Vol","Cum_Avg_Ask_Vol","Cum_Avg_Max_Bid",
               "Cum_Avg_Min_Ask","log_Avg_Ask_Vol","Bid_Int_Ind","Ask_Int_Ind","Upward_Mid","Stationary_Mid",
               "Downward_Mid","Upward_Spread_Cross","Vol_Diff", "Price_Diff",
               "Stationary_Spread_Cross","Downward_Spread_Cross","Ask_Price_Diff","Bid_Price_Diff", "log_Cum_Avg_Max_Bid",
               "log_Cum_Avg_Min_Ask","log_Cum_Avg_Bid_Vol","log_Cum_Avg_Ask_Vol"]
    
    features_df = pd.DataFrame(index = data.index, columns = columns)
    features_df = features_df.fillna(0)
    
    bid_cols = list(headers[2:58:6])
    bid_vol_cols = list(headers[4:60:6])
    ask_cols = list(headers[5:61:6])
    ask_vol_cols = list(headers[7:63:6])
    bids = pd.DataFrame(data[bid_cols])
    bids_vol = pd.DataFrame(data[bid_vol_cols])
    asks = pd.DataFrame(data[ask_cols])
    asks_vol = pd.DataFrame(data[ask_vol_cols])
    
    features_df["Max_Bid"] = bids.max(axis = 1)
    features_df["log_Max_Bid"],features_df["Cum_Avg_Max_Bid"],features_df["log_Cum_Avg_Max_Bid"] = build_features(features_df["Max_Bid"])
    
    features_df["Min_Ask"] = asks.min(axis = 1)
    features_df["log_Min_Ask"],features_df["Cum_Avg_Min_Ask"],features_df["log_Cum_Avg_Min_Ask"] = build_features(features_df["Min_Ask"])
    
    #features_df["Mid_Price"] = features_df.mean(axis = 1)
    features_df["Mid_Price"] = (features_df["Max_Bid"] + features_df["Min_Ask"])/2
    features_df["log_Mid_Price"] = np.log(features_df["Mid_Price"])
    
    features_df["Avg_Bid_Vol"] = bids_vol.mean(axis = 1)
    features_df["log_Avg_Bid_Vol"],features_df["Cum_Avg_Bid_Vol"],features_df["log_Cum_Avg_Bid_Vol"] = build_features(features_df["Avg_Bid_Vol"])
    
    features_df["Avg_Ask_Vol"] = asks_vol.mean(axis = 1)
    features_df["log_Avg_Ask_Vol"],features_df["Cum_Avg_Ask_Vol"],features_df["log_Cum_Avg_Ask_Vol"] = build_features(features_df["Avg_Ask_Vol"])
    
    features_df["Price_Diff"] = features_df["Min_Ask"] - features_df["Max_Bid"]
    features_df["Vol_Diff"] = features_df["Avg_Ask_Vol"] - features_df["Avg_Bid_Vol"]
    #features_df["Bid_Int_Ind"] = pd.DataFrame(features_df["Avg_Bid_Vol"]).diff(axis = 0)
    #features_df["Bid_Int_Ind"] = pd.DataFrame(np.array(features_df["Bid_Int_Ind"] > 0).astype(int))
    #features_df["Ask_Int_Ind"] = pd.DataFrame(features_df["Avg_Ask_Vol"]).diff(axis = 0)
    #features_df["Ask_Int_Ind"] = pd.DataFrame(np.array(features_df["Ask_Int_Ind"] > 0).astype(int))
    
    #shift upward by 1 (e.g. Upward_Mid = 1 if mid price is going to go up)
    features_df["Shift_Mid"] = pd.DataFrame(np.array(np.insert(np.array(features_df["Mid_Price"]),len(data), np.repeat(0, delta_t))[delta_t:len(data)+delta_t]))
    features_df["Mid_Diff"] = features_df["Shift_Mid"] - features_df["Mid_Price"]
    features_df["Mid_Diff"][len(data)-delta_t:len(data)] = 0
    features_df["Upward_Mid"] = pd.DataFrame(np.array(features_df["Mid_Diff"] > 0).astype(int))
    features_df["Stationary_Mid"] = pd.DataFrame(np.array(features_df["Mid_Diff"] == 0).astype(int))
    features_df["Stationary_Mid"][len(data)-delta_t:len(data)] = 0
    features_df["Downward_Mid"] = pd.DataFrame(np.array(features_df["Mid_Diff"] < 0).astype(int))
    
    features_df["Spread_Bid_Col"] = pd.DataFrame(np.array(np.insert(np.array(features_df["Max_Bid"]),len(data), np.repeat(0, delta_t))[delta_t:len(data)+delta_t]))
    features_df["Spread_Ask_Col"] = pd.DataFrame(np.array(np.insert(np.array(features_df["Min_Ask"]),len(data), np.repeat(0, delta_t))[delta_t:len(data)+delta_t]))
    features_df["Past_Bid_Col"] = pd.DataFrame(np.array(np.insert(np.array(features_df["Max_Bid"]),0, np.repeat(0, delta_t))[0:len(data)]))
    features_df["Past_Ask_Col"] = pd.DataFrame(np.array(np.insert(np.array(features_df["Min_Ask"]),0, np.repeat(0, delta_t))[0:len(data)]))
                                                                                                                                       
    features_df["Ask_Price_Diff"] = features_df["Min_Ask"] - features_df["Past_Ask_Col"]
    features_df["Ask_Price_Diff"][0:delta_t] = 0
    features_df["Bid_Price_Diff"] = features_df["Max_Bid"] - features_df["Past_Bid_Col"]
    features_df["Bid_Price_Diff"][0:delta_t] = 0
    
    features_df["Upward_Spread_Cross"] = pd.DataFrame(np.array(features_df["Spread_Bid_Col"] > 
                                                               features_df["Min_Ask"]).astype(int))
    features_df["Upward_Spread_Cross"][len(data)-delta_t:len(data)] = 0
    features_df["Stationary_Spread_Cross"] = pd.DataFrame(np.array((features_df["Spread_Bid_Col"] <=
                                                                    features_df["Min_Ask"]) & 
                                                                   (features_df["Max_Bid"] <=
                                                                    features_df["Spread_Ask_Col"])).astype(int))
    features_df["Stationary_Spread_Cross"][len(data)-delta_t:len(data)] = 0
    features_df["Downward_Spread_Cross"] = pd.DataFrame(np.array(features_df["Max_Bid"] > 
                                                               features_df["Spread_Ask_Col"]).astype(int))
    features_df["Downward_Spread_Cross"][len(data)-delta_t:len(data)] = 0

    log_columns = ["log_Max_Bid","log_Cum_Avg_Max_Bid","Bid_Price_Diff","log_Min_Ask","log_Cum_Avg_Min_Ask",
                   "Ask_Price_Diff","log_Mid_Price","log_Avg_Bid_Vol","log_Cum_Avg_Bid_Vol",
               "log_Avg_Ask_Vol","log_Cum_Avg_Ask_Vol","Price_Diff","Vol_Diff",
               "Upward_Mid","Stationary_Mid","Downward_Mid","Upward_Spread_Cross",
               "Stationary_Spread_Cross","Downward_Spread_Cross"]
    columns = ["Max_Bid","Cum_Avg_Max_Bid","Bid_Price_Diff","Min_Ask","Cum_Avg_Min_Ask","Ask_Price_Diff",
               "Mid_Price","Avg_Bid_Vol","Cum_Avg_Bid_Vol",
               "Avg_Ask_Vol","Cum_Avg_Ask_Vol","Price_Diff","Vol_Diff",
               "Upward_Mid","Stationary_Mid","Downward_Mid","Upward_Spread_Cross",
               "Stationary_Spread_Cross","Downward_Spread_Cross"]
    log_features_df = features_df[log_columns]
    features_df = features_df[columns]
    return(log_features_df,features_df)

# 2. Model Fitting

In [78]:
svm_train_df, rf_train_df = data_processing(train, headers)
rf_train_df.head(10)

,Max_Bid,Cum_Avg_Max_Bid,Bid_Price_Diff,Min_Ask,Cum_Avg_Min_Ask,Ask_Price_Diff,Mid_Price,Avg_Bid_Vol,Cum_Avg_Bid_Vol,Avg_Ask_Vol,Cum_Avg_Ask_Vol,Price_Diff,Vol_Diff,Upward_Mid,Stationary_Mid,Downward_Mid,Upward_Spread_Cross,Stationary_Spread_Cross,Downward_Spread_Cross
0,569.02,569.02,0.00,570.00,570.00,0.00,569.510,100.6,100.6,130.0,130.0,0.98,29.4,1,0,0,0,1,0
1,569.61,569.61,0.59,569.95,569.95,-0.05,569.780,68.1,68.1,125.0,125.0,0.34,56.9,1,0,0,0,1,0
2,569.69,569.69,0.08,569.94,569.94,-0.01,569.815,88.6,88.6,149.7,149.7,0.25,61.1,0,1,0,0,1,0
3,569.69,569.69,0.00,569.94,569.94,0.00,569.815,138.9,138.9,493.0,493.0,0.25,354.1,0,1,0,0,1,0
4,569.69,569.69,0.00,569.94,569.94,0.00,569.815,138.9,138.9,659.6,659.6,0.25,520.7,0,0,1,0,1,0
5,569.61,569.61,-0.08,569.98,569.98,0.04,569.795,148.3,148.3,766.4,766.4,0.37,618.1,1,0,0,0,1,0
6,569.95,569.95,0.34,569.98,569.98,0.00,569.965,253.8,253.8,766.6,766.6,0.03,512.8,0,0,1,0,1,0
7,569.55,569.55,-0.40,569.98,569.98,0.00,569.765,210.6,210.6,767.4,767.4,0.43,556.8,0,0,1,0,1,0
8,569.56,569.56,0.01,569.60,569.60,-0.38,569.580,201.4,201.4,741.7,741.7,0.04,540.3,1,0,0,0,1,0
9,569.56,569.56,0.00,570.00,570.00,0.40,569.780,201.4,201.4,419.4,419.4,0.44,218.0,0,1,0,0,1,0


In [79]:
svm_valid_df, rf_valid_df = data_processing(valid, headers)
svm_valid_df.tail(10)

,log_Max_Bid,log_Cum_Avg_Max_Bid,Bid_Price_Diff,log_Min_Ask,log_Cum_Avg_Min_Ask,Ask_Price_Diff,log_Mid_Price,log_Avg_Bid_Vol,log_Cum_Avg_Bid_Vol,log_Avg_Ask_Vol,log_Cum_Avg_Ask_Vol,Price_Diff,Vol_Diff,Upward_Mid,Stationary_Mid,Downward_Mid,Upward_Spread_Cross,Stationary_Spread_Cross,Downward_Spread_Cross
1959,6.340218,6.340059,-0.02,6.340359,6.340382,0.00,6.340289,5.101694,4.849135,5.102911,5.064176,0.08,0.2,0,0,1,0,1,0
1960,6.339971,6.340084,-0.14,6.340359,6.340382,0.00,6.340165,4.940928,4.839135,5.252273,5.071103,0.22,51.1,1,0,0,0,1,0
1961,6.340024,6.340095,0.03,6.340359,6.340382,0.00,6.340192,4.837868,4.853592,5.072044,5.058473,0.19,33.3,0,0,1,0,1,0
1962,6.339848,6.340096,-0.10,6.340359,6.340382,0.00,6.340104,4.446174,4.863063,5.523459,5.111687,0.29,165.2,1,0,0,0,1,0
1963,6.340059,6.340102,0.12,6.340359,6.340382,0.00,6.340209,5.037602,4.925731,5.416100,5.184028,0.17,70.9,0,1,0,0,1,0
1964,6.340059,6.340086,0.00,6.340359,6.340382,0.00,6.340209,5.130490,4.983196,4.744932,5.149817,0.17,-54.1,1,0,0,0,1,0
1965,6.340165,6.340081,0.06,6.340359,6.340380,0.00,6.340262,5.130490,5.030503,5.044715,5.149933,0.11,-13.9,0,0,1,0,1,0
1966,6.339918,6.340051,-0.14,6.340359,6.340359,0.00,6.340139,5.037602,5.032788,4.946630,5.138970,0.25,-13.4,0,0,1,0,1,0
1967,6.339918,6.340044,0.00,6.340306,6.340354,-0.03,6.340112,5.012633,5.001796,4.342506,5.098524,0.22,-73.4,1,0,0,0,1,0
1968,6.339971,6.340015,0.03,6.340342,6.340352,0.02,6.340156,5.078917,4.992471,4.772378,5.072984,0.21,-42.4,0,0,0,0,0,0


## 1) Upward Mid

In [13]:
######## START WITH RF ########
### don't need to scale data so use the non-log output df[1] ###

In [80]:
cols = ["Max_Bid","Cum_Avg_Max_Bid","Bid_Price_Diff","Min_Ask","Cum_Avg_Min_Ask","Ask_Price_Diff",
               "Mid_Price","Avg_Bid_Vol","Cum_Avg_Bid_Vol",
               "Avg_Ask_Vol","Cum_Avg_Ask_Vol","Price_Diff","Vol_Diff"]
train_df_rf = rf_train_df[cols]

valid_df_rf = rf_valid_df[cols]

In [81]:
def rf_pred(trained, actual):
    valid_rf_pred = trained.predict(valid_df_rf)
    valid_rf_pred = pd.DataFrame(valid_rf_pred)
    result = pd.concat([valid_rf_pred, actual], axis=1)
    result.columns = ['Predicted', actual.name]
    print("Number of predicted", actual.name, ": ", sum(result['Predicted']))
    return(result)

In [82]:
def acc(pred, y, data_name = "validation"):
    count = 0
    for i in range(0,len(pred)):
        if pred[i] == y[i]:
            count = count + 1
        
    accuracy = count / len(pred)
    print("Accuracy on", data_name, "set: ", accuracy, "\n")
    return(accuracy)

In [83]:
def rf_train(X, Y, actual):
    ests = list(np.arange(10,410,100))
    depth = list(np.arange(2,23,2))
    
    for i in range(0,len(ests)):
        for j in range(0,len(depth)):
            random_forest = RandomForestClassifier(n_estimators = ests[i], max_depth = depth[j])
            random_forest = random_forest.fit(X, Y)
            scores = cross_val_score(random_forest, X, Y, cv = 10)
            print("Accuracy =",  scores.mean(), "Trees =", ests[i],"Depth =", depth[j])
            result = rf_pred(random_forest, actual)
            rf_acc = acc(list(result["Predicted"]),list(result[actual.name]))

In [84]:
def rf_choose_param(X, Y, num_est, max_dep):
    random_forest = RandomForestClassifier(n_estimators = num_est, max_depth = max_dep)
    random_forest = random_forest.fit(X, Y)
    return(random_forest)

In [173]:
rf_train(train_df_rf, rf_train_df["Upward_Mid"], rf_valid_df["Upward_Mid"])

Accuracy = 0.566735966736 Trees = 10 Depth = 2
Number of predicted Upward_Mid :  48
Accuracy on validation set:  0.5738953783646521 

Accuracy = 0.55738045738 Trees = 10 Depth = 4
Number of predicted Upward_Mid :  228
Accuracy on validation set:  0.5627221940071102 

Accuracy = 0.539501039501 Trees = 10 Depth = 6
Number of predicted Upward_Mid :  281
Accuracy on validation set:  0.5713560182833926 

Accuracy = 0.535550935551 Trees = 10 Depth = 8
Number of predicted Upward_Mid :  383
Accuracy on validation set:  0.5835449466734383 

Accuracy = 0.537006237006 Trees = 10 Depth = 10
Number of predicted Upward_Mid :  459
Accuracy on validation set:  0.5683087861858812 

Accuracy = 0.505821205821 Trees = 10 Depth = 12
Number of predicted Upward_Mid :  544
Accuracy on validation set:  0.5647536820721178 

Accuracy = 0.491683991684 Trees = 10 Depth = 14
Number of predicted Upward_Mid :  555
Accuracy on validation set:  0.5551041137633317 

Accuracy = 0.508731808732 Trees = 10 Depth = 16
Number

In [85]:
rf_upward_mid = rf_choose_param(train_df_rf, rf_train_df["Upward_Mid"], 110, 2)

In [20]:
### Now SVM for Upward Mid ###

In [86]:
cols = ["log_Max_Bid","log_Cum_Avg_Max_Bid","Bid_Price_Diff","log_Min_Ask","log_Cum_Avg_Min_Ask",
                   "Ask_Price_Diff","log_Mid_Price","log_Avg_Bid_Vol","log_Cum_Avg_Bid_Vol",
               "log_Avg_Ask_Vol","log_Cum_Avg_Ask_Vol","Price_Diff","Vol_Diff"]
train_df_svm = svm_train_df[cols]

valid_df_svm = svm_valid_df[cols]

In [87]:
x = np.array(train_df_svm)
size = len(x)
col_len = len(train_df_svm.columns)
inp = x.reshape(size, col_len)

valid_x = np.array(valid_df_svm)
valid_size = len(valid_x)
valid_col_len = len(valid_df_svm.columns)
valid_inp = valid_x.reshape(valid_size, valid_col_len)

In [88]:
def svm_pred(target, actual):
    gammas = [2**-10, 2**-5, 2, 2**5, 2**10]
    C = [2**-5 ,2, 2**5, 2**10]
    
    for i in range(0,len(gammas)):
        for j in range(0,len(C)):
            svm_param = svm.SVC(kernel = 'rbf', C = C[j], gamma = gammas[i])
            scores = cross_validation.cross_val_score(svm_param, inp, target, cv = 5)
            print("Accuracy =", scores.mean(), "C =", C[j], "gamma =", gammas[i]) 
            
            svm_param.fit(inp, target)
            valid_svm_pred = svm_param.predict(valid_inp)
            valid_svm_pred = pd.DataFrame(valid_svm_pred)
            test_svm = pd.concat([valid_svm_pred, actual], axis=1)
            test_svm.columns = ['Predicted', actual.name]
            print("Number of predicted", actual.name, ": ", sum(test_svm['Predicted']))
            svm_acc = acc(list(test_svm["Predicted"]), list(test_svm[actual.name]))  

In [89]:
def svm_choose_param(target, C_param, gamma_param):
    svm_param = svm.SVC(kernel = 'rbf', C = C_param, gamma = gamma_param)
    svm_param.fit(inp, target)
    return(svm_param)      

In [178]:
svm_pred(svm_train_df["Upward_Mid"], rf_valid_df["Upward_Mid"])

Accuracy = 0.567567567568 C = 0.03125 gamma = 0.0009765625
Number of predicted Upward_Mid :  0
Accuracy on validation set:  0.5678009141696293 

Accuracy = 0.554677754678 C = 2 gamma = 0.0009765625
Number of predicted Upward_Mid :  124
Accuracy on validation set:  0.5627221940071102 

Accuracy = 0.554677754678 C = 32 gamma = 0.0009765625
Number of predicted Upward_Mid :  232
Accuracy on validation set:  0.5606907059421026 

Accuracy = 0.549896049896 C = 1024 gamma = 0.0009765625
Number of predicted Upward_Mid :  310
Accuracy on validation set:  0.5566277298120873 

Accuracy = 0.567567567568 C = 0.03125 gamma = 0.03125
Number of predicted Upward_Mid :  0
Accuracy on validation set:  0.5678009141696293 

Accuracy = 0.541787941788 C = 2 gamma = 0.03125
Number of predicted Upward_Mid :  338
Accuracy on validation set:  0.5454545454545454 

Accuracy = 0.528274428274 C = 32 gamma = 0.03125
Number of predicted Upward_Mid :  488
Accuracy on validation set:  0.5200609446419502 

Accuracy = 0.51

In [90]:
svm_upward_mid = svm_choose_param(svm_train_df["Upward_Mid"], 1024, 1024)

## 2) Stationary Mid

In [26]:
###### Now DT for Stationary Mid #####

In [19]:
rf_train(train_df_rf, rf_train_df["Stationary_Mid"], rf_valid_df["Stationary_Mid"])

Accuracy = 0.8322684019 Trees = 10 Depth = 2
Number of predicted Stationary_Mid :  0
Accuracy on validation set:  0.8781107160995429 

Accuracy = 0.825625075482 Trees = 10 Depth = 4
Number of predicted Stationary_Mid :  0
Accuracy on validation set:  0.8781107160995429 

Accuracy = 0.823135448926 Trees = 10 Depth = 6
Number of predicted Stationary_Mid :  0
Accuracy on validation set:  0.8781107160995429 

Accuracy = 0.81877211995 Trees = 10 Depth = 8
Number of predicted Stationary_Mid :  0
Accuracy on validation set:  0.8781107160995429 

Accuracy = 0.82354822645 Trees = 10 Depth = 10
Number of predicted Stationary_Mid :  0
Accuracy on validation set:  0.8781107160995429 

Accuracy = 0.814415277071 Trees = 10 Depth = 12
Number of predicted Stationary_Mid :  0
Accuracy on validation set:  0.8781107160995429 

Accuracy = 0.812749478452 Trees = 10 Depth = 14
Number of predicted Stationary_Mid :  1
Accuracy on validation set:  0.877602844083291 

Accuracy = 0.813787690899 Trees = 10 Depth 

In [91]:
rf_stationary_mid =  rf_choose_param(train_df_rf, rf_train_df["Stationary_Mid"], 310, 2)

In [120]:
###### Now SVM for Stationary Mid #####

In [30]:
svm_pred(svm_train_df["Stationary_Mid"], rf_valid_df["Stationary_Mid"])

Accuracy = 0.911850489997 C = 0.03125 gamma = 0.0009765625
Number of predicted Stationary_Mid :  0
Accuracy on validation set:  0.8781107160995429 

Accuracy = 0.91081185251 C = 2 gamma = 0.0009765625
Number of predicted Stationary_Mid :  0
Accuracy on validation set:  0.8781107160995429 

Accuracy = 0.908525812877 C = 32 gamma = 0.0009765625
Number of predicted Stationary_Mid :  0
Accuracy on validation set:  0.8781107160995429 

Accuracy = 0.908316833229 C = 1024 gamma = 0.0009765625
Number of predicted Stationary_Mid :  1
Accuracy on validation set:  0.8786185881157949 

Accuracy = 0.911850489997 C = 0.03125 gamma = 0.03125
Number of predicted Stationary_Mid :  0
Accuracy on validation set:  0.8781107160995429 

Accuracy = 0.910188150988 C = 2 gamma = 0.03125
Number of predicted Stationary_Mid :  0
Accuracy on validation set:  0.8781107160995429 

Accuracy = 0.90083328705 C = 32 gamma = 0.03125
Number of predicted Stationary_Mid :  4
Accuracy on validation set:  0.8760792280345353 


In [92]:
svm_stationary_mid = svm_choose_param(svm_train_df["Stationary_Mid"], 1024, 1024)

## 3) Downward Mid

In [28]:
######## RF Downward Mid ########

In [28]:
rf_train(train_df_rf, rf_train_df["Downward_Mid"], rf_valid_df["Downward_Mid"])

Accuracy = 0.526613934993 Trees = 10 Depth = 2
Number of predicted Downward_Mid :  564
Accuracy on validation set:  0.5698324022346368 

Accuracy = 0.510197959889 Trees = 10 Depth = 4
Number of predicted Downward_Mid :  604
Accuracy on validation set:  0.5698324022346368 

Accuracy = 0.52704961964 Trees = 10 Depth = 6
Number of predicted Downward_Mid :  848
Accuracy on validation set:  0.563737938039614 

Accuracy = 0.497105809129 Trees = 10 Depth = 8
Number of predicted Downward_Mid :  935
Accuracy on validation set:  0.5540883697308279 

Accuracy = 0.509359439834 Trees = 10 Depth = 10
Number of predicted Downward_Mid :  874
Accuracy on validation set:  0.5525647536820721 

Accuracy = 0.481931189488 Trees = 10 Depth = 12
Number of predicted Downward_Mid :  893
Accuracy on validation set:  0.5449466734382935 

Accuracy = 0.489415629322 Trees = 10 Depth = 14
Number of predicted Downward_Mid :  925
Accuracy on validation set:  0.5175215845606908 

Accuracy = 0.492309820194 Trees = 10 Dep

In [93]:
rf_downward_mid =  rf_choose_param(train_df_rf, rf_train_df["Downward_Mid"], 310, 2)

In [30]:
###### Now SVM for Downward Mid #####

In [31]:
svm_pred(svm_train_df["Downward_Mid"], rf_valid_df["Downward_Mid"])

Accuracy = 0.52079002079 C = 0.03125 gamma = 0.0009765625
Number of predicted Downward_Mid :  0
Accuracy on validation set:  0.5545962417470798 

Accuracy = 0.490436590437 C = 2 gamma = 0.0009765625
Number of predicted Downward_Mid :  338
Accuracy on validation set:  0.5352971051295073 

Accuracy = 0.4841995842 C = 32 gamma = 0.0009765625
Number of predicted Downward_Mid :  348
Accuracy on validation set:  0.537328593194515 

Accuracy = 0.504365904366 C = 1024 gamma = 0.0009765625
Number of predicted Downward_Mid :  739
Accuracy on validation set:  0.531742001015744 

Accuracy = 0.52079002079 C = 0.03125 gamma = 0.03125
Number of predicted Downward_Mid :  0
Accuracy on validation set:  0.5545962417470798 

Accuracy = 0.5 C = 2 gamma = 0.03125
Number of predicted Downward_Mid :  801
Accuracy on validation set:  0.5286947689182326 

Accuracy = 0.514760914761 C = 32 gamma = 0.03125
Number of predicted Downward_Mid :  910
Accuracy on validation set:  0.49974606399187405 

Accuracy = 0.5072

In [94]:
svm_downward_mid = svm_choose_param(svm_train_df["Downward_Mid"], 1024, 1024)

## 4) Upward Spread Crossing

In [32]:
##### RF for Upward Spread Crossing #########

In [33]:
rf_train(train_df_rf, rf_train_df["Upward_Spread_Cross"], rf_valid_df["Upward_Spread_Cross"])

Accuracy = 0.98981416942 Trees = 10 Depth = 2
Number of predicted Upward_Spread_Cross :  0
Accuracy on validation set:  0.9954291518537328 

Accuracy = 0.97114197025 Trees = 10 Depth = 4
Number of predicted Upward_Spread_Cross :  0
Accuracy on validation set:  0.9954291518537328 

Accuracy = 0.97134943913 Trees = 10 Depth = 6
Number of predicted Upward_Spread_Cross :  0
Accuracy on validation set:  0.9954291518537328 

Accuracy = 0.986702136225 Trees = 10 Depth = 8
Number of predicted Upward_Spread_Cross :  0
Accuracy on validation set:  0.9954291518537328 

Accuracy = 0.98379757191 Trees = 10 Depth = 10
Number of predicted Upward_Spread_Cross :  0
Accuracy on validation set:  0.9954291518537328 

Accuracy = 0.973631596806 Trees = 10 Depth = 12
Number of predicted Upward_Spread_Cross :  1
Accuracy on validation set:  0.9949212798374809 

Accuracy = 0.987116642656 Trees = 10 Depth = 14
Number of predicted Upward_Spread_Cross :  0
Accuracy on validation set:  0.9954291518537328 

Accurac

In [95]:
rf_upward_spread =  rf_choose_param(train_df_rf, rf_train_df["Upward_Spread_Cross"], 10, 16)

In [35]:
####### SVM for Upward Spread Crossing ###########

In [36]:
svm_pred(svm_train_df["Upward_Spread_Cross"], rf_valid_df["Upward_Spread_Cross"])

Accuracy = 0.99002121371 C = 0.03125 gamma = 0.0009765625
Number of predicted Upward_Spread_Cross :  0
Accuracy on validation set:  0.9954291518537328 

Accuracy = 0.99002121371 C = 2 gamma = 0.0009765625
Number of predicted Upward_Spread_Cross :  0
Accuracy on validation set:  0.9954291518537328 

Accuracy = 0.99002121371 C = 32 gamma = 0.0009765625
Number of predicted Upward_Spread_Cross :  0
Accuracy on validation set:  0.9954291518537328 

Accuracy = 0.987113633232 C = 1024 gamma = 0.0009765625
Number of predicted Upward_Spread_Cross :  0
Accuracy on validation set:  0.9954291518537328 

Accuracy = 0.99002121371 C = 0.03125 gamma = 0.03125
Number of predicted Upward_Spread_Cross :  0
Accuracy on validation set:  0.9954291518537328 

Accuracy = 0.98939816075 C = 2 gamma = 0.03125
Number of predicted Upward_Spread_Cross :  0
Accuracy on validation set:  0.9954291518537328 

Accuracy = 0.987112985118 C = 32 gamma = 0.03125
Number of predicted Upward_Spread_Cross :  1
Accuracy on valid

In [96]:
svm_upward_spread = svm_choose_param(svm_train_df["Upward_Spread_Cross"], 1024, 1024)

## 5) Stationary Spread Crossing

In [38]:
##### RF for Stationary Spread Crossing #########

In [39]:
rf_train(train_df_rf, rf_train_df["Stationary_Spread_Cross"], rf_valid_df["Stationary_Spread_Cross"])

Accuracy = 0.958666167764 Trees = 10 Depth = 2
Number of predicted Stationary_Spread_Cross :  1969
Accuracy on validation set:  0.9873031995937024 

Accuracy = 0.957420923157 Trees = 10 Depth = 4
Number of predicted Stationary_Spread_Cross :  1969
Accuracy on validation set:  0.9873031995937024 

Accuracy = 0.957420923157 Trees = 10 Depth = 6
Number of predicted Stationary_Spread_Cross :  1969
Accuracy on validation set:  0.9873031995937024 

Accuracy = 0.955138332356 Trees = 10 Depth = 8
Number of predicted Stationary_Spread_Cross :  1968
Accuracy on validation set:  0.9867953275774505 

Accuracy = 0.951401304552 Trees = 10 Depth = 10
Number of predicted Stationary_Spread_Cross :  1966
Accuracy on validation set:  0.9857795835449467 

Accuracy = 0.9491178475 Trees = 10 Depth = 12
Number of predicted Stationary_Spread_Cross :  1969
Accuracy on validation set:  0.9873031995937024 

Accuracy = 0.948907359322 Trees = 10 Depth = 14
Number of predicted Stationary_Spread_Cross :  1967
Accura

In [97]:
rf_stationary_spread =  rf_choose_param(train_df_rf, rf_train_df["Stationary_Spread_Cross"], 310, 2)

In [41]:
####### SVM for Stationary Spread Crossing ###########

In [42]:
svm_pred(svm_train_df["Stationary_Spread_Cross"], rf_valid_df["Stationary_Spread_Cross"])

Accuracy = 0.976299582303 C = 0.03125 gamma = 0.0009765625
Number of predicted Stationary_Spread_Cross :  1969
Accuracy on validation set:  0.9873031995937024 

Accuracy = 0.973184317505 C = 2 gamma = 0.0009765625
Number of predicted Stationary_Spread_Cross :  1969
Accuracy on validation set:  0.9873031995937024 

Accuracy = 0.972976633186 C = 32 gamma = 0.0009765625
Number of predicted Stationary_Spread_Cross :  1969
Accuracy on validation set:  0.9873031995937024 

Accuracy = 0.969235724772 C = 1024 gamma = 0.0009765625
Number of predicted Stationary_Spread_Cross :  1969
Accuracy on validation set:  0.9873031995937024 

Accuracy = 0.976299582303 C = 0.03125 gamma = 0.03125
Number of predicted Stationary_Spread_Cross :  1969
Accuracy on validation set:  0.9873031995937024 

Accuracy = 0.974637675968 C = 2 gamma = 0.03125
Number of predicted Stationary_Spread_Cross :  1969
Accuracy on validation set:  0.9873031995937024 

Accuracy = 0.970689515011 C = 32 gamma = 0.03125
Number of predi

In [98]:
svm_stationary_spread = svm_choose_param(svm_train_df["Stationary_Spread_Cross"], 1024, 1024)

## 6) Downward Spread Crossing

In [44]:
##### RF for Downward Spread Crossing #########

In [32]:
rf_train(train_df_rf, rf_train_df["Downward_Spread_Cross"], rf_valid_df["Downward_Spread_Cross"])

Accuracy = 0.969476141079 Trees = 10 Depth = 2
Number of predicted Downward_Spread_Cross :  0
Accuracy on validation set:  0.9923819197562215 

Accuracy = 0.969268672199 Trees = 10 Depth = 4
Number of predicted Downward_Spread_Cross :  0
Accuracy on validation set:  0.9923819197562215 

Accuracy = 0.967193983402 Trees = 10 Depth = 6
Number of predicted Downward_Spread_Cross :  0
Accuracy on validation set:  0.9923819197562215 

Accuracy = 0.966779045643 Trees = 10 Depth = 8
Number of predicted Downward_Spread_Cross :  0
Accuracy on validation set:  0.9923819197562215 

Accuracy = 0.962837136929 Trees = 10 Depth = 10
Number of predicted Downward_Spread_Cross :  0
Accuracy on validation set:  0.9923819197562215 

Accuracy = 0.966779045643 Trees = 10 Depth = 12
Number of predicted Downward_Spread_Cross :  2
Accuracy on validation set:  0.9913661757237177 

Accuracy = 0.966778181189 Trees = 10 Depth = 14
Number of predicted Downward_Spread_Cross :  0
Accuracy on validation set:  0.99238191

In [99]:
rf_downward_spread =  rf_choose_param(train_df_rf, rf_train_df["Downward_Spread_Cross"], 310, 2)

In [46]:
####### SVM for Downward Spread Crossing ###########

In [47]:
svm_pred(svm_train_df["Downward_Spread_Cross"], rf_valid_df["Downward_Spread_Cross"])

Accuracy = 0.986486486486 C = 0.03125 gamma = 0.0009765625
Number of predicted Downward_Spread_Cross :  0
Accuracy on validation set:  0.9923819197562215 

Accuracy = 0.983367983368 C = 2 gamma = 0.0009765625
Number of predicted Downward_Spread_Cross :  0
Accuracy on validation set:  0.9923819197562215 

Accuracy = 0.98316008316 C = 32 gamma = 0.0009765625
Number of predicted Downward_Spread_Cross :  0
Accuracy on validation set:  0.9923819197562215 

Accuracy = 0.981081081081 C = 1024 gamma = 0.0009765625
Number of predicted Downward_Spread_Cross :  0
Accuracy on validation set:  0.9923819197562215 

Accuracy = 0.986486486486 C = 0.03125 gamma = 0.03125
Number of predicted Downward_Spread_Cross :  0
Accuracy on validation set:  0.9923819197562215 

Accuracy = 0.985446985447 C = 2 gamma = 0.03125
Number of predicted Downward_Spread_Cross :  0
Accuracy on validation set:  0.9923819197562215 

Accuracy = 0.983367983368 C = 32 gamma = 0.03125
Number of predicted Downward_Spread_Cross :  1

In [100]:
svm_downward_spread = svm_choose_param(svm_train_df["Downward_Spread_Cross"], 1024, 1024)

# 3. Model Assessment 

In [101]:
#figure out test to get output from all rows
svm_test_df, rf_test_df = data_processing(test, headers, delta_t=30)
cols = ["Max_Bid","Cum_Avg_Max_Bid","Bid_Price_Diff","Min_Ask","Cum_Avg_Min_Ask","Ask_Price_Diff",
               "Mid_Price","Avg_Bid_Vol","Cum_Avg_Bid_Vol",
               "Avg_Ask_Vol","Cum_Avg_Ask_Vol","Price_Diff","Vol_Diff"]
test_df_rf = rf_test_df[cols]

log_cols = ["log_Max_Bid","log_Cum_Avg_Max_Bid","Bid_Price_Diff","log_Min_Ask","log_Cum_Avg_Min_Ask",
                   "Ask_Price_Diff","log_Mid_Price","log_Avg_Bid_Vol","log_Cum_Avg_Bid_Vol",
               "log_Avg_Ask_Vol","log_Cum_Avg_Ask_Vol","Price_Diff","Vol_Diff"]
test_df_svm = svm_test_df[log_cols]

In [102]:
def performance(param, model, sub_test_df, test_df):
    test_pred = model.predict(sub_test_df)
    test_pred = pd.DataFrame(test_pred)
    test_pred.columns = ["Predicted"]
    print("Number of predicted", param, ":", sum(test_pred["Predicted"]), "\n",
          "Length of prediction vector: ", len(test_pred), "\n",
          "Number of true", param, ":", sum(test_df[param])) 
          
    test_acc = acc(list(test_pred["Predicted"]),list(test_df[param]), "test")

    #get metrics
    y_true = np.array(test_df[param])
    y_pred = np.array(test_pred)
    
    precision, recall, fscore = precision_recall_fscore_support(y_true, y_pred)[0:3]
    print("Precision: ", precision, "; Recall: ",recall, "; Fscore: ", fscore, "\n", "\n")

In [103]:
performance("Upward_Mid", rf_upward_mid, test_df_rf, rf_test_df)
performance("Stationary_Mid", rf_stationary_mid, test_df_rf, rf_test_df) 
performance("Downward_Mid", rf_downward_mid, test_df_rf, rf_test_df) 
performance("Upward_Spread_Cross", rf_upward_spread, test_df_rf, rf_test_df) 
performance("Stationary_Spread_Cross", rf_stationary_spread, test_df_rf, rf_test_df) 
performance("Downward_Spread_Cross", rf_downward_spread, test_df_rf, rf_test_df)

Number of predicted Upward_Mid : 333 
 Length of prediction vector:  129324 
 Number of true Upward_Mid : 52731
Accuracy on test set:  0.5924499706164362 

Precision:  [ 0.59259173  0.53753754] ; Recall:  [ 0.99798937  0.00339459] ; Fscore:  [ 0.74362791  0.00674657] 
 

Number of predicted Stationary_Mid : 0 
 Length of prediction vector:  129324 
 Number of true Stationary_Mid : 17502
Accuracy on test set:  0.8646654913241162 

Precision:  [ 0.86466549  0.        ] ; Recall:  [ 1.  0.] ; Fscore:  [ 0.92742156  0.        ] 
 

Number of predicted Downward_Mid : 39012 
 Length of prediction vector:  129324 
 Number of true Downward_Mid : 59061
Accuracy on test set:  0.5806114874269277 

Precision:  [ 0.58872575  0.56182713] ; Recall:  [ 0.75671406  0.37110784] ; Fscore:  [ 0.6622326   0.44697317] 
 

Number of predicted Upward_Spread_Cross : 0 
 Length of prediction vector:  129324 
 Number of true Upward_Spread_Cross : 743
Accuracy on test set:  0.9942547400327859 

Precision:  [ 0.99

In [104]:
performance("Upward_Mid", svm_upward_mid, test_df_svm, svm_test_df)
performance("Stationary_Mid", svm_stationary_mid, test_df_svm, svm_test_df) 
performance("Downward_Mid", svm_downward_mid, test_df_svm, svm_test_df) 
performance("Upward_Spread_Cross", svm_upward_spread, test_df_svm, svm_test_df) 
performance("Stationary_Spread_Cross", svm_stationary_spread, test_df_svm, svm_test_df) 
performance("Downward_Spread_Cross", svm_downward_spread, test_df_svm, svm_test_df)

Number of predicted Upward_Mid : 1 
 Length of prediction vector:  129324 
 Number of true Upward_Mid : 52731
Accuracy on test set:  0.5922489251801677 

Precision:  [ 0.5922535  0.       ] ; Recall:  [ 0.99998694  0.        ] ; Fscore:  [ 0.74391499  0.        ] 
 

Number of predicted Stationary_Mid : 0 
 Length of prediction vector:  129324 
 Number of true Stationary_Mid : 17502
Accuracy on test set:  0.8646654913241162 

Precision:  [ 0.86466549  0.        ] ; Recall:  [ 1.  0.] ; Fscore:  [ 0.92742156  0.        ] 
 

Number of predicted Downward_Mid : 1 
 Length of prediction vector:  129324 
 Number of true Downward_Mid : 59061
Accuracy on test set:  0.5433175589991031 

Precision:  [ 0.54331403  1.        ] ; Recall:  [  1.00000000e+00   1.69316469e-05] ; Fscore:  [  7.04087461e-01   3.38627205e-05] 
 

Number of predicted Upward_Spread_Cross : 0 
 Length of prediction vector:  129324 
 Number of true Upward_Spread_Cross : 743
Accuracy on test set:  0.9942547400327859 

Precis

# 4. Interpretations

In [105]:
## set aside test set 11:00-12:30
test = data[data["Time"] >= '2012/05/22 11:00:00.000']
test = test.reset_index(drop = True)
test.head()
len(test)

129324

In [106]:
## training set
train = data30[data30["Time"] < '2012/05/22 11:00:00.000']
train = train.reset_index(drop = True)
train.tail()
len(train)

6779

In [107]:
## get headers for indexing later
headers = list(data.columns.values)
print(headers[5:61:6])

['TVITCH_41::AAPL.ASK_PRICE1..TVITCH_41__AAPL_3', 'TVITCH_41::AAPL.ASK_PRICE2..TVITCH_41__AAPL_3', 'TVITCH_41::AAPL.ASK_PRICE3..TVITCH_41__AAPL_3', 'TVITCH_41::AAPL.ASK_PRICE4..TVITCH_41__AAPL_3', 'TVITCH_41::AAPL.ASK_PRICE5..TVITCH_41__AAPL_3', 'TVITCH_41::AAPL.ASK_PRICE6..TVITCH_41__AAPL_3', 'TVITCH_41::AAPL.ASK_PRICE7..TVITCH_41__AAPL_3', 'TVITCH_41::AAPL.ASK_PRICE8..TVITCH_41__AAPL_3', 'TVITCH_41::AAPL.ASK_PRICE9..TVITCH_41__AAPL_3', 'TVITCH_41::AAPL.ASK_PRICE10..TVITCH_41__AAPL_3']


In [108]:
logit_train_df = data_processing(train, headers)[0]
cols = ["log_Max_Bid","log_Cum_Avg_Max_Bid","Bid_Price_Diff","log_Min_Ask","log_Cum_Avg_Min_Ask",
                   "Ask_Price_Diff","log_Mid_Price","log_Avg_Bid_Vol","log_Cum_Avg_Bid_Vol",
               "log_Avg_Ask_Vol","log_Cum_Avg_Ask_Vol","Price_Diff","Vol_Diff"]
train_df_logit = logit_train_df[cols]
train_df_logit.head()
len(train_df_logit)

6779

In [109]:
logit_test_df = data_processing(test, headers, delta_t=30)[0]
test_df_logit = logit_test_df[cols]
test_df_logit.head()
len(test_df_logit)

129324

In [110]:
def logit_train(X, Y):
    model = LogisticRegression()
    model = model.fit(X, Y)

    # check the accuracy on the training set
    print("Accuracy on training set: ", model.score(X, Y))
    print("Accuracy if predicted all zeros: ", 1 - Y.mean())
    coef = pd.DataFrame(list(zip(X.columns, np.transpose(model.coef_))))
    exp_coef = pd.DataFrame(np.exp(np.array(coef[1]).astype(float)))
    coef = pd.concat([coef, exp_coef], axis=1)
    coef.columns = ["Variables", "Log Coefficients", "Odds Ratio"]
    return(model, coef)

In [111]:
def logit_pred(model, test_df, actual):
    predicted = model.predict(test_df)
    print("Number of predicted", actual.name, ": ", sum(predicted))
    print("Accuracy on test set: ", metrics.accuracy_score(actual, predicted))
    print("Confusion matrix: \n", metrics.confusion_matrix(actual, predicted))
    print("Classification report: \n", metrics.classification_report(actual, predicted))

In [112]:
model_upward_mid, coef_upward_mid = logit_train(train_df_logit, logit_train_df["Upward_Mid"])
logit_pred(model_upward_mid, test_df_logit, logit_test_df["Upward_Mid"])
coef_upward_mid

Accuracy on training set:  0.570880660864
Accuracy if predicted all zeros:  0.5676353444460835
Number of predicted Upward_Mid :  6647
Accuracy on test set:  0.590331261019
Confusion matrix: 
 [[73145  3448]
 [49532  3199]]
Classification report: 
              precision    recall  f1-score   support

          0       0.60      0.95      0.73     76593
          1       0.48      0.06      0.11     52731

avg / total       0.55      0.59      0.48    129324



,Variables,Log Coefficients,Odds Ratio
0,log_Max_Bid,[0.0119086713835],1.011980
1,log_Cum_Avg_Max_Bid,[-0.0119479456363],0.988123
2,Bid_Price_Diff,[1.48253595768],4.404100
3,log_Min_Ask,[0.0120151652103],1.012088
4,log_Cum_Avg_Min_Ask,[-0.0101953601539],0.989856
5,Ask_Price_Diff,[1.19156135959],3.292218
6,log_Mid_Price,[0.0119620329459],1.012034
7,log_Avg_Bid_Vol,[-0.32834377087],0.720115
8,log_Cum_Avg_Bid_Vol,[0.135971960501],1.145650
9,log_Avg_Ask_Vol,[0.137679838808],1.147608


In [113]:
model_stationary_mid, coef_stationary_mid = logit_train(train_df_logit, logit_train_df["Stationary_Mid"])
logit_pred(model_stationary_mid, test_df_logit, logit_test_df["Stationary_Mid"])
coef_stationary_mid

Accuracy on training set:  0.902050449919
Accuracy if predicted all zeros:  0.9020504499188671
Number of predicted Stationary_Mid :  0
Accuracy on test set:  0.864665491324
Confusion matrix: 
 [[111822      0]
 [ 17502      0]]
Classification report: 
              precision    recall  f1-score   support

          0       0.86      1.00      0.93    111822
          1       0.00      0.00      0.00     17502

avg / total       0.75      0.86      0.80    129324



,Variables,Log Coefficients,Odds Ratio
0,log_Max_Bid,[-0.0834538603097],0.919934
1,log_Cum_Avg_Max_Bid,[-0.0814136144278],0.921812
2,Bid_Price_Diff,[0.219149930926],1.245018
3,log_Min_Ask,[-0.0833884970086],0.919994
4,log_Cum_Avg_Min_Ask,[-0.0823438145909],0.920955
5,Ask_Price_Diff,[0.442794947886],1.557053
6,log_Mid_Price,[-0.0834213471618],0.919963
7,log_Avg_Bid_Vol,[0.492271682523],1.636029
8,log_Cum_Avg_Bid_Vol,[-0.201998258801],0.817096
9,log_Avg_Ask_Vol,[-0.240581444327],0.786171


In [114]:
model_downward_mid, coef_downward_mid = logit_train(train_df_logit, logit_train_df["Downward_Mid"])
logit_pred(model_downward_mid, test_df_logit, logit_test_df["Downward_Mid"])
coef_downward_mid

Accuracy on training set:  0.550376161676
Accuracy if predicted all zeros:  0.5304617200177018
Number of predicted Downward_Mid :  23403
Accuracy on test set:  0.557406204571
Confusion matrix: 
 [[59473 10790]
 [46448 12613]]
Classification report: 
              precision    recall  f1-score   support

          0       0.56      0.85      0.68     70263
          1       0.54      0.21      0.31     59061

avg / total       0.55      0.56      0.51    129324



,Variables,Log Coefficients,Odds Ratio
0,log_Max_Bid,[-0.0121613143547],0.987912
1,log_Cum_Avg_Max_Bid,[0.000403374694646],1.000403
2,Bid_Price_Diff,[-1.51892103847],0.218948
3,log_Min_Ask,[-0.0122707226325],0.987804
4,log_Cum_Avg_Min_Ask,[-0.000119336095584],0.999881
5,Ask_Price_Diff,[-1.3988354781],0.246884
6,log_Mid_Price,[-0.0122159973929],0.987858
7,log_Avg_Bid_Vol,[0.122892327489],1.130763
8,log_Cum_Avg_Bid_Vol,[-0.0657121009514],0.936400
9,log_Avg_Ask_Vol,[-0.0364211721515],0.964234


In [115]:
model_upward_spread, coef_upward_spread = logit_train(train_df_logit, logit_train_df["Upward_Spread_Cross"])
logit_pred(model_upward_spread, test_df_logit, logit_test_df["Upward_Spread_Cross"])
coef_upward_spread

Accuracy on training set:  0.991591680189
Accuracy if predicted all zeros:  0.9915916801888184
Number of predicted Upward_Spread_Cross :  0
Accuracy on test set:  0.994254740033
Confusion matrix: 
 [[128581      0]
 [   743      0]]
Classification report: 
              precision    recall  f1-score   support

          0       0.99      1.00      1.00    128581
          1       0.00      0.00      0.00       743

avg / total       0.99      0.99      0.99    129324



,Variables,Log Coefficients,Odds Ratio
0,log_Max_Bid,[-0.222464143839],0.800544
1,log_Cum_Avg_Max_Bid,[-0.224863897609],0.798625
2,Bid_Price_Diff,[0.393175842455],1.481679
3,log_Min_Ask,[-0.22775221139],0.796322
4,log_Cum_Avg_Min_Ask,[-0.225833311241],0.797851
5,Ask_Price_Diff,[-1.01646000464],0.361874
6,log_Mid_Price,[-0.225108593753],0.798430
7,log_Avg_Bid_Vol,[0.377880350287],1.459188
8,log_Cum_Avg_Bid_Vol,[-0.455946815677],0.633848
9,log_Avg_Ask_Vol,[-0.326662880069],0.721327


In [116]:
model_stationary_spread, coef_stationary_spread = logit_train(train_df_logit, logit_train_df["Stationary_Spread_Cross"])
logit_pred(model_stationary_spread, test_df_logit, logit_test_df["Stationary_Spread_Cross"])
coef_stationary_spread

Accuracy on training set:  0.979643015194
Accuracy if predicted all zeros:  0.020356984806018552
Number of predicted Stationary_Spread_Cross :  129324
Accuracy on test set:  0.989151278958
Confusion matrix: 
 [[     0   1403]
 [     0 127921]]
Classification report: 
              precision    recall  f1-score   support

          0       0.00      0.00      0.00      1403
          1       0.99      1.00      0.99    127921

avg / total       0.98      0.99      0.98    129324



,Variables,Log Coefficients,Odds Ratio
0,log_Max_Bid,[0.196955158477],1.217689
1,log_Cum_Avg_Max_Bid,[0.199246296555],1.220483
2,Bid_Price_Diff,[-0.557333055295],0.572734
3,log_Min_Ask,[0.207219741529],1.230253
4,log_Cum_Avg_Min_Ask,[0.201431404019],1.223152
5,Ask_Price_Diff,[1.43774642586],4.211195
6,log_Mid_Price,[0.202088148962],1.223956
7,log_Avg_Bid_Vol,[-0.34178056083],0.710504
8,log_Cum_Avg_Bid_Vol,[0.265197761794],1.303689
9,log_Avg_Ask_Vol,[-0.0690391199362],0.933290


In [117]:
model_downward_spread, coef_downward_spread = logit_train(train_df_logit, logit_train_df["Downward_Spread_Cross"])
logit_pred(model_downward_spread, test_df_logit, logit_test_df["Downward_Spread_Cross"])
coef_downward_spread

Accuracy on training set:  0.988198849388
Accuracy if predicted all zeros:  0.9881988493878153
Number of predicted Downward_Spread_Cross :  0
Accuracy on test set:  0.995128514429
Confusion matrix: 
 [[128694      0]
 [   630      0]]
Classification report: 
              precision    recall  f1-score   support

          0       1.00      1.00      1.00    128694
          1       0.00      0.00      0.00       630

avg / total       0.99      1.00      0.99    129324



,Variables,Log Coefficients,Odds Ratio
0,log_Max_Bid,[-0.23542963471],0.790231
1,log_Cum_Avg_Max_Bid,[-0.236107713909],0.789696
2,Bid_Price_Diff,[0.462455755668],1.587969
3,log_Min_Ask,[-0.243598484036],0.783802
4,log_Cum_Avg_Min_Ask,[-0.238175706662],0.788064
5,Ask_Price_Diff,[-1.1414331194],0.319361
6,log_Mid_Price,[-0.239514602909],0.787010
7,log_Avg_Bid_Vol,[0.185531628642],1.203858
8,log_Cum_Avg_Bid_Vol,[-0.0279716437646],0.972416
9,log_Avg_Ask_Vol,[0.510695313973],1.666449


# 5. Trading strategies

In [118]:
## set aside test set 11:00-12:30
test = data[(data["Time"] >= '2012/05/22 11:00:00.000') & (data["Time"] <= '2012/05/22 12:00:00.000')]
test = test.reset_index(drop = True)
test.head()
len(test)

76307

In [119]:
def rf_pred_test(trained, test, name):
    rf_pred = pd.DataFrame(trained.predict(test))
    rf_pred.columns = ["Predicted_" + name]
    return(rf_pred)

In [120]:
def trading(test, delta_t):
    headers = list(test.columns.values)
    cols = ["Max_Bid","Cum_Avg_Max_Bid","Bid_Price_Diff","Min_Ask","Cum_Avg_Min_Ask","Ask_Price_Diff",
               "Mid_Price","Avg_Bid_Vol","Cum_Avg_Bid_Vol",
               "Avg_Ask_Vol","Cum_Avg_Ask_Vol","Price_Diff","Vol_Diff"]
    test = data_processing(test, headers, delta_t = delta_t)[1][cols]
    upward_mid = rf_pred_test(rf_upward_mid, test, "Upward_Mid")
    stationary_mid = rf_pred_test(rf_stationary_mid, test, "Stationary_Mid")
    downward_mid = rf_pred_test(rf_downward_mid, test, "Downward_Mid")
    test = pd.concat([test, upward_mid, stationary_mid, downward_mid], axis=1)
    print(upward_mid.sum(), "\n")
    print(stationary_mid.sum(), "\n")
    print(downward_mid.sum(), "\n")

    total_profit=0; i=0
    while (i + delta_t) < len(test):
        if test["Predicted_Upward_Mid"][i] == 1:
            total_profit = total_profit + test["Max_Bid"][i+delta_t] - test["Min_Ask"][i]
        elif test["Predicted_Downward_Mid"][i] == 1:
            total_profit = total_profit + test["Max_Bid"][i] - test["Min_Ask"][i+delta_t]
        i = i + delta_t + 1 
    print("Total profit: ", total_profit)
    return(total_profit)

In [121]:
def trading_break_assump(test, delta_t):
    headers = list(test.columns.values)
    cols = ["Max_Bid","Cum_Avg_Max_Bid","Bid_Price_Diff","Min_Ask","Cum_Avg_Min_Ask","Ask_Price_Diff",
               "Mid_Price","Avg_Bid_Vol","Cum_Avg_Bid_Vol",
               "Avg_Ask_Vol","Cum_Avg_Ask_Vol","Price_Diff","Vol_Diff"]
    test = data_processing(test, headers, delta_t = delta_t)[1][cols]
    upward_mid = rf_pred_test(rf_upward_mid, test, "Upward_Mid")
    stationary_mid = rf_pred_test(rf_stationary_mid, test, "Stationary_Mid")
    downward_mid = rf_pred_test(rf_downward_mid, test, "Downward_Mid")
    test = pd.concat([test, upward_mid, stationary_mid, downward_mid], axis=1)
    print(upward_mid.sum(), "\n")
    print(stationary_mid.sum(), "\n")
    print(downward_mid.sum(), "\n")
    
    future_actions={}
    total_profit=0
    for i in range(len(test)):
        if i in future_actions.keys():
            if future_actions[i]=="Buy":
                total_profit -= test["Min_Ask"][i] #downward cross; buy share to return 
                del future_actions[i]
            else:
                total_profit += test["Max_Bid"][i] #upward cross; sell share 
                del future_actions[i]
        elif test["Predicted_Upward_Mid"][i] == 1:
            total_profit -= test["Min_Ask"][i]
            future_actions[i+delta_t]="Sell"
        elif test["Predicted_Downward_Mid"][i] == 1:
            total_profit += test["Max_Bid"][i]
            future_actions[i+delta_t]="Buy" 
    print("Total profit: ", total_profit)
    return(total_profit)

In [125]:
total_profit=trading(test,delta_t=30)

Predicted_Upward_Mid    156
dtype: int64 

Predicted_Stationary_Mid    0
dtype: int64 

Predicted_Downward_Mid    22602
dtype: int64 

Total profit:  -125.64


In [123]:
# Passive strategy is better than an active strategy when you aren't predicting for every value 
total_profit=trading_break_assump(test,delta_t=30)

Predicted_Upward_Mid    156
dtype: int64 

Predicted_Stationary_Mid    0
dtype: int64 

Predicted_Downward_Mid    22602
dtype: int64 

Total profit:  -2818.77


# 6. Plots

In [126]:
#prepare dataset for mid price plot (code in R)
plot_data = pd.concat([data["Time"], data_processing(data, headers)[1]["Mid_Price"]], axis = 1)
len(plot_data) 

332673

In [115]:
plot_data.to_csv("AAPL_mid_price.csv", index = False)

In [127]:
#accuracy
plt.style.use('ggplot')
N = 6
rf = (.5953, .8647, .5770, .9942, .9892, .9951)

ind = np.arange(6)  # the x locations for the groups
width = 0.22       # the width of the bars

fig = plt.figure()
ax = fig.add_subplot(111)
rects1 = ax.bar(ind+.18, rf, width, color="#FF6666")

svm = (.5922, .8647, .5433, .9943, .9892, .9951)
rects2 = ax.bar(ind+width+.18, svm, width, color="#619CFF")

logistic = (.5903, .8447, .5573, .9943, .9892, .9951)
rects3 = ax.bar(ind+2*width+.18, logistic, width, color="#33CC00")

# add some
ax.set_ylabel('Test Set Accuracies')
ax.set_title('Model Accuracy Comparison')
ax.set_xticks(ind+1.5*width+.18)
ax.tick_params(top='off',right='off')
ax.set_xticklabels( ('Upward Mid', 'Stationary Mid', 'Downward Mid', 
                     'Upward Cross', 'Stationary Cross', 'Downward Cross'), 
                  rotation=45)

ax.legend( (rects1[0], rects2[0], rects3[0]), ('RF', 'SVM', 'Logistic'), loc=4)
plt.gcf().subplots_adjust(bottom=0.2)
plt.savefig('./modelaccuracy.png', dpi=300)
plt.show()